In [1]:
from tasks.VSTtask import VSTtask
from agents.LLMagent import LLMagent
from manager.TaskManager import TaskManager

In [2]:
YOUR_ANTHROPIC_API_KEY=""
YOUR_OPENAI_API_KEY = ""

# To use OpenAI:
#agent = LLMagent(model_name="gpt-4o-mini", openai_api_key="YOUR_OPENAI_API_KEY")

# To use Anthropic:
agent = LLMagent(model_name="claude-3-5-sonnet-latest", anthropic_api_key=YOUR_ANTHROPIC_API_KEY)

# To use a local model:
#agent = LLMagent(model_name="marcelbinz/Llama-3.1-Centaur-8B-adapter")


Using Anthropic API for GPT model


In [4]:
manager = TaskManager(n_simulations=10, n_rounds=10, num_quadrants=2, num_cues=1, pipe=agent, verbose=True)
metrics = manager.run_simulations()


Simulation 1/10

=== Task Description ===
You will play a game with 10 rounds.
In each round you'll see active cues (chooseable):
One quadrant has 90% one color/10% other
Other quadrants have 50/50 color distribution
At least one cue active per round
Active cues disappear after random duration

After 10 rounds, identify the biased quadrant.
Correct: +100 points, Wrong: -100 points.

=== Beginning Rounds ===

--- Round 1 ---

Accumulated prompt shown to LLM:
--------------------
You will play a game with 10 rounds.
In each round you'll see active cues (chooseable):
One quadrant has 90% one color/10% other
Other quadrants have 50/50 color distribution
At least one cue active per round
Active cues disappear after random duration

After 10 rounds, identify the biased quadrant.
Correct: +100 points, Wrong: -100 points.

Round 1: Available cues A. Based on previous observations, choose one cue by responding with just the letter. You press <<
--------------------

LLM chose: A
Result: RED
(c

In [5]:
print(metrics)

{'n_rounds': 10, 'n_cues': 1, 'n_simulations': 10, 'success_rate': 0.0, 'quadrant_distribution': {'quadrant_1': {'times_chosen': 0, 'times_correct': 7}, 'quadrant_2': {'times_chosen': 0, 'times_correct': 3}}, 'timestamp': '20250208_184028'}
